# Miscellaneous 

Some topics don't need an entire chapter but are still worth discussing

## Async Eliding

Async eliding is where you don't await a task but pass it through as a return value.  
https://blog.stephencleary.com/2016/12/eliding-async-await.html  

This can greatly reduce CPU cycles in some cases and is important around guard clauses.

Consider this example:

In [16]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public async Task<int> DoSomeCalculations(int a) {

    var result = await AddAsync(a, 1);
    return result;
}

var sum = await DoSomeCalculations(3);
Console.WriteLine(sum);

4


You don't need to await AddAsync here. As discussed in chapter 4, this will generate an entire state machine for this async await while you don't really need it.  
You can just as easily do this:

In [17]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public Task<int> DoSomeCalculations(int a) {

    return AddAsync(a, 1);
}

var sum = await DoSomeCalculations(3);
Console.WriteLine(sum);

4


This has the same result but doesn't generate a state machine for this method, you can see that when you paste both examples into sharplab.io.

This example only works if you're awaiting a single task though, so it's not applicable to this example:  

In [ ]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public async Task<int> DoSomeCalculations(int a) {

    var result = await AddAsync(a, 1);
    result += await AddAsync(a, 4);
    result += await AddAsync(a, 8);

    return result;
}

var sum = await DoSomeCalculations(3);
Console.WriteLine(sum);

22


This eliding technique becomes particularly important when working with guard clauses.  
Consider the following example:

In [18]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public async Task<int> DoSomeCalculations(int a, int? b) {

    if (b is null) throw new ArgumentNullException(nameof(b));

    var result = await AddAsync(a, 1);
    result += await AddAsync(a, b.Value);
    result += await AddAsync(a, 8);

    return result;
}

var sum = await DoSomeCalculations(3, 9);
Console.WriteLine(sum);

27


This looks fine, however if we now pass null as the `b` parameter this throws an exception.

In [19]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public async Task<int> DoSomeCalculations(int a, int? b) {

    if (b is null) throw new ArgumentNullException(nameof(b));

    var result = await AddAsync(a, 1);
    result += await AddAsync(a, b.Value);
    result += await AddAsync(a, 8);

    return result;
}

try {
    var sum = await DoSomeCalculations(3, null);
    Console.WriteLine(sum);
} 
catch (Exception ex) 
{
    Console.WriteLine(ex.Message);
}

Value cannot be null. (Parameter 'b')


Since we are awaiting this the state machine will neatly handle the exception and re-throw it inside of your method.  
If we don't await this something else happens.

In [20]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public async Task<int> DoSomeCalculations(int a, int? b) {

    if (b is null) throw new ArgumentNullException(nameof(b));

    var result = await AddAsync(a, 1);
    result += await AddAsync(a, b.Value);
    result += await AddAsync(a, 8);

    return result;
}

try {
    var sum = DoSomeCalculations(3, null);
    Console.WriteLine(sum);
} 
catch (Exception ex) 
{
    Console.WriteLine(ex.Message);
}

System.Threading.Tasks.Task`1[System.Int32]


Because your async method is not awaited, it throws the exception in the state machine, which is outside of your method's context so it bubbles up to your application and crashes the process. This is dotnet intentional behavior, if the runtime encounters an unhandled exception it terminates the process.

So how do you prevent that?
Like so:

In [9]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public Task<int> DoSomeCalculationsAsync(int a, int? b) {

    if (b is null) throw new ArgumentNullException(nameof(b));

    return DoSomeCalculations(a, b.Value);
}

private async Task<int> DoSomeCalculations(int a, int b) {

    var result = await AddAsync(a, 1);
    result += await AddAsync(a, b);
    result += await AddAsync(a, 8);

    return result;
}

try {
    var sum = DoSomeCalculationsAsync(3, null);
    Console.WriteLine(sum);
} 
catch (Exception ex) 
{
    Console.WriteLine(ex.Message);
}

Value cannot be null. (Parameter 'b')


As you can see, this throws the exception within your context, so it doesn't blow up the application while it's functionally the same.

So why is this important? You're supposed to `await` them anyway right?
The short answer, deferred await.
For example a `Task.WhenAll()` scenario.

In [21]:
using System.Threading.Tasks;

Task<int>AddAsync(int a, int b) {
    return Task.FromResult(a + b);
}

public Task<int> DoSomeCalculationsAsync(int a, int? b) {

    if (b is null) throw new ArgumentNullException(nameof(b));

    return DoSomeCalculations(a, b.Value);
}

private async Task<int> DoSomeCalculations(int a, int b) {

    var result = await AddAsync(a, 1);
    result += await AddAsync(a, b);
    result += await AddAsync(a, 8);

    return result;
}

try {

    var sumTask1 = DoSomeCalculationsAsync(3, 6);
    var sumTask2 = DoSomeCalculationsAsync(3, null);

    await Task.WaitAll(sumTask1, sumTask2);

    Console.WriteLine(sumTask1.GetAwaiter().GetResult());
    Console.WriteLine(sumTask1.GetAwaiter().GetResult());
} 
catch (Exception ex) 
{
    Console.WriteLine(ex.Message);
}

Value cannot be null. (Parameter 'b')
